In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "drive/My Drive/Projects/3D_BB"

/content/drive/My Drive/Projects/3D_BB


In [ ]:
import os
import sys
import cv2
import torch
from torchvision.models import vgg
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision.models import vgg19_bn

In [ ]:
def get_image(name):
    img = cv2.imread(name, cv2.IMREAD_COLOR).astype(np.float) / 255
    img[:, :, 0] = (img[:, :, 0] - 0.406) / 0.225
    img[:, :, 1] = (img[:, :, 1] - 0.456) / 0.224
    img[:, :, 2] = (img[:, :, 2] - 0.485) / 0.229
    return img

def get_confident(bins=3, overlap=25/180.0 * np.pi, local_angle = 45*np.pi/180):

    
    centerAngle = np.zeros(bins)
    interval = 2 * np.pi / bins
    
    for i in range(1, bins):
        centerAngle[i] = i * interval
        confidence = np.zeros(bins)
    confidence_multi = np.zeros(bins)
    for i in range(bins):
        diff = abs(centerAngle[i] - local_angle)
        if diff > np.pi:
            diff = 2 * np.pi - diff
        if diff <= interval / 2 + overlap:
            confidence_multi[i] = 1
        if diff < interval / 2:
            confidence[i] = 1
    angleDiff = local_angle - centerAngle
    
    return confidence_multi, angleDiff

In [ ]:
import os

data = []
labels = []
confidents = []
anglediffs = []
directory = 'orientation_angle_images/'
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        fname = os.path.join(directory, filename)
        image = get_image(fname)
        crop_image = image[190:380,245:700]
        crop_image = cv2.resize(src=crop_image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC) #224, 224, 3
        crop_image = np.transpose(crop_image, (2,0,1))
        label = float(filename.split('_')[1].split('.png')[0])
        confident_multi, anglediff = get_confident(local_angle=label)
        
        data.append(crop_image)
        labels.append(label) 
        confidents.append(confident_multi)
        anglediffs.append(anglediff)
    else:
        continue

In [ ]:
def OrientationLoss(orient, angleDiff, confGT):
    #
    # orid = [sin(delta), cos(delta)] shape = [batch, bins, 2]
    # angleDiff = GT - center, shape = [batch, bins]
    #
    [batch, _, bins] = orient.size()
    cos_diff = torch.cos(angleDiff)
    sin_diff = torch.sin(angleDiff)
    cos_ori = orient[:, :, 0]
    sin_ori = orient[:, :, 1]
    mask1 = (confGT != 0)
    mask2 = (confGT == 0)
    count = torch.sum(mask1, dim=1)
    tmp = cos_diff * cos_ori + sin_diff * sin_ori
    tmp[mask2] = 0
    total = torch.sum(tmp, dim = 1)
    count = count.type(torch.FloatTensor).cuda()
    total = total / count
    return -torch.sum(total) / batch

class OModel(nn.Module):
    def __init__(self, features=None, bins=2, w = 0.4):
        super(OModel, self).__init__()
        self.bins = bins
        self.w = w
        self.features = features
        self.orientation = nn.Sequential(
                    nn.Linear(512 * 7 * 7, 256),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(256, 256),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(256, bins*2) # to get sin and cos
                )
        self.confidence = nn.Sequential(
                    nn.Linear(512 * 7 * 7, 256),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(256, 256),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(256, bins),
                    nn.Softmax()
                    #nn.Sigmoid()
                )

    def forward(self, x):
        x = self.features(x) # 512 x 7 x 7
        x = x.view(-1, 512 * 7 * 7)
        orientation = self.orientation(x)
        orientation = orientation.view(-1, self.bins, 2)
        orientation = F.normalize(orientation, dim=2)
        confidence = self.confidence(x)
        return orientation, confidence

In [ ]:
vgg = vgg19_bn(pretrained=True)
model = OModel(features=vgg.features, bins=3).cuda()

In [ ]:
tensor_train_w = torch.from_numpy(np.array(data)).float()
train_label_angle = np.asarray(labels)
train_label_confident = np.asarray(confidents)
train_label_angle_diff = np.asarray(anglediffs)

In [ ]:
bs = 10
all_indices = torch.randperm(tensor_train_w.size(0)).split(bs)

In [ ]:
opt_SGD = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
conf_LossFunc = nn.CrossEntropyLoss()

In [ ]:
epochs=1000
for epoch in range(epochs):

    loss_epoch = 0
    model.train()
    for batch_ndx in all_indices:

        input_ = tensor_train_w[batch_ndx].cuda()
        label_angl = train_label_angle[batch_ndx]
        label_conf = train_label_confident[batch_ndx]
        label_anglediff = train_label_angle_diff[batch_ndx]
        
        confidence_arg = np.argmax(label_conf, axis = 1)
        confidence_arg = Variable(torch.LongTensor(confidence_arg.astype(np.int)), requires_grad=False).cuda()
        angleDiff = Variable(torch.FloatTensor(label_anglediff), requires_grad=False).cuda()
        confidence_multi = Variable(torch.LongTensor(label_conf.astype(np.int)), requires_grad=False).cuda()
        
        orient, conf = model(input_)
        
        conf_loss = conf_LossFunc(conf, confidence_arg)
        orient_loss = OrientationLoss(orient, angleDiff, confidence_multi)
        
        loss_theta = conf_loss + 1 * orient_loss
        
        opt_SGD.zero_grad()
        loss_theta.backward()
        opt_SGD.step()
        loss_epoch += loss_theta.item()
    print('loss= ', loss_epoch)
    print('conf loss: ', conf_loss.item())
    print('orient_loss loss: ', orient_loss.item())

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss=  8.358046770095825
conf loss:  1.080615758895874
orient_loss loss:  -0.16138856112957
loss=  4.96262127161026
conf loss:  1.0814446210861206
orient_loss loss:  -0.7376042604446411
loss=  2.9003578424453735
conf loss:  1.089975118637085
orient_loss loss:  -0.7207928895950317
loss=  2.347559094429016
conf loss:  1.0791027545928955
orient_loss loss:  -0.7860393524169922
loss=  2.2918747663497925
conf loss:  1.0648736953735352
orient_loss loss:  -0.7628414034843445
loss=  2.4492491483688354
conf loss:  1.0652215480804443
orient_loss loss:  -0.8038789629936218
loss=  1.9986843466758728
conf loss:  1.048027515411377
orient_loss loss:  -0.8205596804618835
loss=  1.8884192109107971
conf loss:  1.0538647174835205
orient_loss loss:  -0.8888848423957825
loss=  1.7220305800437927
conf loss:  1.04958176612854
orient_loss loss:  -0.8645797967910767
loss=  1.4286304116249084
conf loss:  1.0573824644088745
orient_loss loss:  -0.8719337582588196
loss=  1.3935924172401428
conf loss:  1.05370306968

KeyboardInterrupt: ignored

In [ ]:
model.eval()
test = tensor_train_w[[0,1]].cuda()
label = train_label_angle[[0,1]]
orient, conf = model(test)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
orient = orient.cpu().data.numpy()
conf = conf.cpu().data.numpy()

argmax = np.argmax(conf)
orient = orient[argmax, :]
cos = orient[0]
sin = orient[1]

theta = np.arctan2(sin, cos) 
print(theta/(np.pi/180))
print(label/(np.pi/180))

[ 38.46667 -59.51578]
[ 24.38515765 -50.72987984]
